## Project 2 - Casimir Funk



In [1]:
#!pip install -r requirements.txt
import pandas as pd
import numpy as np

### 1.1 #A Description of population of interest.

Population of San Francisco and comparing the difference in minimum cost diets of people across different diet groups, such as vegetarian, pescatarian, vegan, meat eater, etc.

### 1.2 #A Dietary Reference Intakes

Write a function that takes as arguments the characterists of a person (e.g. age, sex) and returns a pandas.Series of Dietary Reference Intakes (DRI's) or "Recommended Daily Allowances" (RDA) of a variety of nutrients appropriate for your population of interest.

In [2]:
def DRI(age=19, sex='M'):
    bmin = pd.read_csv('./diet_minimums.csv').set_index('Nutrition').iloc[:,2:]
    bmax = pd.read_csv('./diet_maximums.csv').set_index('Nutrition').iloc[:,2:]
    combined_df = pd.concat([bmin,-bmax])
    if sex == "M":
        if age <=3:
            return combined_df.loc[:, 'C 1-3']
        if age>=4 and age<=8:
            return combined_df.loc[:, 'M 4-8']
        if age>=9 and age<=13:
            return combined_df.loc[:, 'M 9-13']
        if age>= 14 and age<=18:
            return combined_df.loc[:,'M 14-18']
        if age>=19 and age <= 30: 
            return combined_df.loc[:, 'M 19-30']
        if age>=31 and age<=50:
            return combined_df.loc[:, 'M 31-50']
        if age>=51:
            return combined_df.loc[:, 'M 51+']
    if sex == "F":
        if age <=3:
            return combined_df.loc[:, 'C 1-3']
        if age>=4 and age<=8:
            return combined_df.loc[:, 'F 4-8']
        if age>=9 and age<=13:
            return combined_df.loc[:, 'F 9-13']
        if age>= 14 and age<=18:
            return combined_df.loc[:,'F 14-18']
        if age>=19 and age <= 30: 
            return combined_df.loc[:, 'F 19-30']
        if age>=31 and age<=50:
            return combined_df.loc[:, 'F 31-50']
        if age>=51:
            return combined_df.loc[:, 'F 51+']
DRI(25, 'M')

Nutrition
Energy                            2400.0
Protein                             56.0
Fiber, total dietary                33.6
Folate, DFE                        400.0
Calcium, Ca                       1000.0
Carbohydrate, by difference        130.0
Iron, Fe                             8.0
Magnesium, Mg                      400.0
Niacin                              16.0
Phosphorus, P                      700.0
Potassium, K                      4700.0
Riboflavin                           1.3
Thiamin                              1.2
Vitamin A, RAE                     900.0
Vitamin B-12                         2.4
Vitamin B-6                          1.3
Vitamin C, total ascorbic acid      90.0
Vitamin E (alpha-tocopherol)        15.0
Vitamin K (phylloquinone)          120.0
Zinc, Zn                            11.0
Sodium, Na                       -2300.0
Name: M 19-30, dtype: float64

### 1.3 #A Data on prices for different foods (Google Sheets)

API user and keys:

In [3]:
user = "project2-casimir-funk"

# API key for Gov
apikey = "V0mOAdVrSineT4d2VTMTJUNPAsBTnjzUAU6e6H6V" # inIyO1begWSRqsYtxS7m6p09PSyq7Qiw7fxzV2qN"

# File with private keys for relevant service account to authenticate
# and access google spreadsheets
!gpg --passphrase "casimir" -d project2-casimir-funk-306520-ae3703c35882.json.gpg > project2-casimir-funk-306520-ae3703c35882.json

serviceacct = {'project2-casimir-funk':'project2-casimir-funk-306520-ae3703c35882.json'}
sheet_url = "https://docs.google.com/spreadsheets/d/1XqnsamAhXq2xnktB_rJ8h8ovvpFQdWWcDkx_1Yrfqdg/edit#gid=0"


gpg: AES256 encrypted data
gpg: encrypted with 1 passphrase


#### Reading data and diet types from google sheet

In [25]:
# sheet: diet type
diet_type = 'mcdonalds'

In [26]:
from eep153_tools import read_sheets

# Accessing google sheets and convert to pandas.dataframe
data = read_sheets(key=sheet_url, json_creds=serviceacct[user], sheet=diet_type)
headers = data.iloc[0]
df  = pd.DataFrame(data.values[1:], columns=headers)
df

,Category,Food,Quantity,Units,Price,FDC,link to price
0,McDonald's,Big Mac,7.60,ounces,6.21,170720,McDonald's App
1,,McChicken,131.00,grams,3.21,173305,McDonald's App
2,,10 pc McNuggets,159.00,grams,5.74,173297,McDonald's App
3,,Double Quarter Pounder w Cheese,280.00,grams,8.04,172066,McDonald's App
4,,Medium Fries,117.00,grams,3.67,170721,McDonald's App
5,,Filet o Fish,134.00,grams,6.20,170319,McDonald's App
6,,Cheeseburger,119.00,grams,2.03,170320,McDonald's App
7,,Hamburger,95.00,grams,2.03,170717,McDonald's App


In [35]:
import fooddatacentral as fdc

# Query to fdc and get dataframe containing nutritional data of foods using their fdc_ids
food_to_nutrients = {}
for food in df['Food'].tolist():
    try:
        fdc_id = int((df[df['Food'] == food].FDC).iloc[0])
        food_to_nutrients[food] = fdc.nutrients(apikey, fdc_id).Quantity
    except AttributeError: 
        warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))

D = pd.DataFrame(food_to_nutrients,dtype=float)
D

,Big Mac,McChicken,10 pc McNuggets,Double Quarter Pounder w Cheese,Medium Fries,Filet o Fish,Cheeseburger,Hamburger
10:0,NaN,0.006,0.006,NaN,0.011,0.045,NaN,0.010
12:0,NaN,0.004,0.005,NaN,0.005,0.048,NaN,0.008
14:0,NaN,0.024,0.035,NaN,0.012,0.164,NaN,0.218
14:1,NaN,0.006,0.010,NaN,0.001,0.015,NaN,0.066
15:0,NaN,0.004,0.006,NaN,0.005,0.019,NaN,0.038
...,...,...,...,...,...,...,...,...
Vitamin K (Dihydrophylloquinone),NaN,NaN,NaN,NaN,48.600,0.000,NaN,NaN
Vitamin K (phylloquinone),NaN,NaN,NaN,NaN,16.000,4.900,NaN,NaN
Vitamins and Other Components,0.00,0.000,0.000,0.00,0.000,0.000,0.00,0.000
Water,51.30,46.110,47.130,50.34,36.630,45.760,45.00,44.530


In [42]:

D.add(1)

,Big Mac,McChicken,10 pc McNuggets,Double Quarter Pounder w Cheese,Medium Fries,Filet o Fish,Cheeseburger,Hamburger
10:0,NaN,1.006,1.006,NaN,1.011,1.045,NaN,1.010
12:0,NaN,1.004,1.005,NaN,1.005,1.048,NaN,1.008
14:0,NaN,1.024,1.035,NaN,1.012,1.164,NaN,1.218
14:1,NaN,1.006,1.010,NaN,1.001,1.015,NaN,1.066
15:0,NaN,1.004,1.006,NaN,1.005,1.019,NaN,1.038
...,...,...,...,...,...,...,...,...
Vitamin K (Dihydrophylloquinone),NaN,NaN,NaN,NaN,49.600,1.000,NaN,NaN
Vitamin K (phylloquinone),NaN,NaN,NaN,NaN,17.000,5.900,NaN,NaN
Vitamins and Other Components,1.00,1.000,1.000,1.00,1.000,1.000,1.00,1.000
Water,52.30,47.110,48.130,51.34,37.630,46.760,46.00,45.530


### 1.4 #A Nutritional content of different foods

For all the foods you're considering you'll need to be able to describe their nutritional content, in terms that allow you to compare with recommended daily allowances. We'll provide points to US government recommendations, but many other countries have their own recommendations, which may be better suited to their populations.

In [28]:
bmin = pd.read_csv('./diet_minimums.csv').set_index('Nutrition').iloc[:,2:]
bmax = pd.read_csv('./diet_maximums.csv').set_index('Nutrition').iloc[:,2:]

In [29]:
import fooddatacentral as fdc

# Convert data type to float
df['Quantity'] = pd.to_numeric(df['Quantity'], downcast="float") 
df['Price'] = pd.to_numeric(df['Price'], downcast="float")

# Convert food quantities to FDC units
df['FDC Quantity'] = df[['Quantity', 'Units']].T.apply(lambda x: fdc.units(x['Quantity'], x['Units']))
df['FDC Price'] = df['Price']/df['FDC Quantity']
df.dropna()

# Get series of food with their price/hectogram
p = df.groupby('Food')['FDC Price'].min()

p

Food
10 pc McNuggets                    3.6100627491309205 / hectogram
Big Mac                             2.882254050016064 / hectogram
Cheeseburger                       1.7058823288989668 / hectogram
Double Quarter Pounder w Cheese    2.8714285578046526 / hectogram
Filet o Fish                         4.62686552930234 / hectogram
Hamburger                           2.136842075147127 / hectogram
McChicken                          2.4503817085091395 / hectogram
Medium Fries                        3.136752201960637 / hectogram
Name: FDC Price, dtype: object

In [30]:
# Remove price/hectogram and drop NAN values
p = p.apply(lambda x:x.magnitude).dropna()

# Compile list that we have both prices and nutritional info for; drop if either missing
selection = list(set(p.index.tolist()).intersection(D.columns.tolist()))
p = p[selection]

p

Food
Medium Fries                       3.136752
McChicken                          2.450382
Filet o Fish                       4.626866
10 pc McNuggets                    3.610063
Cheeseburger                       1.705882
Double Quarter Pounder w Cheese    2.871429
Hamburger                          2.136842
Big Mac                            2.882254
Name: FDC Price, dtype: float64

In [32]:
# Drop nutritional information for foods we don't know the price of,
# and replace missing nutrients NAN with zeros.
Aall = D[p.index].fillna(0)

# Drop rows of A that we don't have constraints for.
Amin = Aall.loc[bmin.index]
Amax = Aall.loc[bmax.index]

# Minimum requirements involve multiplying constraint by -1 to make <=.
A = pd.concat([Amin, -Amax])
A

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['Folate, DFE'], dtype='object', name='Nutrition'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [11]:
b = pd.concat([bmin, -bmax])
b

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,
Energy,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
"Fiber, total dietary",14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
"Folate, DFE",150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
"Calcium, Ca",700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0
"Carbohydrate, by difference",130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0
"Iron, Fe",7.0,10.0,10.0,8.0,8.0,15.0,11.0,18.0,8.0,18.0,8.0,8.0,8.0
"Magnesium, Mg",80.0,130.0,130.0,240.0,240.0,360.0,410.0,310.0,400.0,320.0,420.0,320.0,420.0
Niacin,6.0,8.0,8.0,12.0,12.0,14.0,16.0,14.0,16.0,14.0,16.0,14.0,16.0


### 1.5 #A Solution Diet

In [12]:
from  scipy.optimize import linprog

sex_age_group = 'M 19-30'

# negate A and b to convert (Ax <= b) to (Ax >= b)
result = linprog(p, -A, -b[sex_age_group], method='interior-point')
result

     con: array([], dtype=float64)
     fun: 6.484349038551782
 message: 'Optimization terminated successfully.'
     nit: 21
   slack: array([1.68468060e-02, 6.30935141e-07, 4.00099016e+01, 2.50472783e+02,
       9.75340993e-06, 1.67951964e+02, 3.21270585e+01, 7.65795074e+01,
       3.31659106e+01, 1.45313596e+02, 2.72704268e+02, 3.31090590e+00,
       1.04500410e+01, 1.39001977e+03, 1.07650149e+01, 4.49319149e+00,
       1.94652073e+02, 1.29526256e-08, 1.14775862e+03, 3.12547275e+00,
       1.52534662e-05])
  status: 0
 success: True
       x: array([1.05293058e-08, 9.71216884e-09, 2.21287332e-07, 1.47475709e-08,
       5.30987489e-09, 1.24528106e-08, 1.50761316e-08, 1.17951108e-01,
       8.13525769e-09, 7.05630110e-09, 1.72999061e-08, 2.42227169e-08,
       1.67002821e+01, 1.70561302e-08, 1.12534721e-08, 7.51259528e-08,
       1.92939881e-07, 8.00554989e-09, 8.63240038e-09, 2.05565640e-07,
       5.61139078e-09, 6.57509588e-09, 3.73650566e-08, 1.32667228e-09,
       2.06038595e-08,

What is the minimum cost diet for the population you're concerned with? How much does it cost, and of what does it consist? How does it vary with age, sex, and level of activity?

In [13]:
print("Cost of %s diet for %s is $%4.2f per day." % (diet_type, sex_age_group, result.fun))

Cost of omnivore diet for M 19-30 is $6.48 per day.


In [14]:
diet = pd.Series(result.x,index=p.index)
outcome_df = pd.DataFrame({"Outcome":np.abs(A).dot(diet),"Recommendation for %s" % (sex_age_group):np.abs(b[sex_age_group])})
excess = outcome_df.diff(axis=1).iloc[:,1]
print("Diet in hectograms")
print(diet[diet >= 1e-6]) #smaller than this are treated as zero
print("\nCost of a %s diet for %s is $%4.2f per day." % (diet_type, sex_age_group, result.fun))
print("\nConstraining nutrients are:")
print(excess.loc[np.abs(excess) < 1e-6].index.tolist())
outcome_df

Diet in hectograms
Food
Pork Liver          0.117951
Canned Tomatoes    16.700282
Wheat Cereal        1.816238
Navy Beans          1.151028
Spinach             2.535386
dtype: float64

Cost of a omnivore diet for M 19-30 is $6.48 per day.

Constraining nutrients are:
['Protein', 'Vitamin E (alpha-tocopherol)']


,Outcome,Recommendation for M 19-30
Nutrition,,
Energy,2400.016847,2400.0
Protein,56.000001,56.0
"Fiber, total dietary",73.609902,33.6
"Folate, DFE",650.472783,400.0
"Calcium, Ca",1000.000010,1000.0
"Carbohydrate, by difference",297.951964,130.0
"Iron, Fe",40.127059,8.0
"Magnesium, Mg",476.579507,400.0
Niacin,49.165911,16.0


### Minimum Cost Diet Function (Combining the above)

In [15]:
# API
user = "project2-casimir-funk"
apikey = "V0mOAdVrSineT4d2VTMTJUNPAsBTnjzUAU6e6H6V" # inIyO1begWSRqsYtxS7m6p09PSyq7Qiw7fxzV2qN"

# File with private keys for relevant service account to authenticate
# and access google spreadsheets
!gpg --passphrase "casimir" -d project2-casimir-funk-306520-ae3703c35882.json.gpg > project2-casimir-funk-306520-ae3703c35882.json

serviceacct = {'project2-casimir-funk':'project2-casimir-funk-306520-ae3703c35882.json'}
sheet_url = "https://docs.google.com/spreadsheets/d/1XqnsamAhXq2xnktB_rJ8h8ovvpFQdWWcDkx_1Yrfqdg/edit#gid=0"

gpg: AES256 encrypted data
gpg: encrypted with 1 passphrase


In [23]:
import pandas as pd
import numpy as np
from eep153_tools import read_sheets
import fooddatacentral as fdc
from  scipy.optimize import linprog



def minimum_cost_diet(diet_type, sex_age_group):
    bmin = pd.read_csv('./diet_minimums.csv').set_index('Nutrition').iloc[:,2:]
    bmax = pd.read_csv('./diet_maximums.csv').set_index('Nutrition').iloc[:,2:]
    
    # Accessing the chosen diet from google sheets and convert to pandas.dataframe
    data = read_sheets(key=sheet_url, json_creds=serviceacct[user], sheet=diet_type)
    headers = data.iloc[0]
    df  = pd.DataFrame(data.values[1:], columns=headers)

    # Convert data type to float
    df['Quantity'] = pd.to_numeric(df['Quantity'], downcast="float") 
    df['Price'] = pd.to_numeric(df['Price'], downcast="float")

    # Convert food quantities to FDC units
    df['FDC Quantity'] = df[['Quantity', 'Units']].T.apply(lambda x: fdc.units(x['Quantity'], x['Units']))
    df['FDC Price'] = df['Price']/df['FDC Quantity']
    df.dropna()

    # Prices of food
    p = df.groupby('Food')['FDC Price'].min()
    p = p.apply(lambda x:x.magnitude).dropna()

    # Query to fdc and get dataframe containing nutritional data of foods using their fdc_ids
    food_to_nutrients = {}
    for food in df['Food'].tolist():
        try:
            fdc_id = int((df[df['Food'] == food].FDC).iloc[0])
            food_to_nutrients[food] = fdc.nutrients(apikey, fdc_id).Quantity
        except AttributeError: 
            warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))
    nutritional_df = pd.DataFrame(food_to_nutrients,dtype=float)
    
    # Drop nutritional information for foods we don't know the price of, and replace missing nutrients with zeros.
    Aall = nutritional_df[p.index].fillna(0)

    # Drop rows of A that we don't have constraints for.
    Amin = Aall.loc[bmin.index]
    Amax = Aall.loc[bmax.index]

    # Negate Sodium values to fit constraint
    A = pd.concat([Amin,-Amax])

    # Solving the diet problem
    b = pd.concat([bmin, -bmax])
    result = linprog(p, -A, -b[sex_age_group], method='interior-point')
    
    if result.success:
        diet = pd.Series(result.x,index=p.index)
        outcome_df = pd.DataFrame({"Outcome":np.abs(A).dot(diet),"Recommendation for %s" % (sex_age_group):np.abs(b[sex_age_group])})
        excess = outcome_df.diff(axis=1).iloc[:,1]
        print("Diet in hectograms")
        print(diet[diet >= 1e-6]) #smaller than this are treated as zero
        print("\nCost of a %s diet for %s is $%4.2f per day." % (diet_type, sex_age_group, result.fun))
        print("\nConstraining nutrients are:")
        print(excess.loc[np.abs(excess) < 1e-6].index.tolist())
        return outcome_df
        
    else:
        for nutrition, value in A.sum(axis=1).items():
            if value == 0:
                print(f"Fail to find solution.\n{diet_type} diet does not provide any {nutrition}!")

### Calling the function

More info on scipy linprog: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html

In [17]:
minimum_cost_diet(diet_type='omnivore', sex_age_group='M 19-30')

Diet in hectograms
Food
Canned Tomatoes    16.700282
Navy Beans          1.151028
Pork Liver          0.117951
Spinach             2.535386
Wheat Cereal        1.816238
dtype: float64

Cost of a omnivore diet for M 19-30 is $6.48 per day.

Constraining nutrients are:
['Protein', 'Vitamin E (alpha-tocopherol)']


,Outcome,Recommendation for M 19-30
Nutrition,,
Energy,2400.016847,2400.0
Protein,56.000001,56.0
"Fiber, total dietary",73.609902,33.6
"Folate, DFE",650.472783,400.0
"Calcium, Ca",1000.000010,1000.0
"Carbohydrate, by difference",297.951964,130.0
"Iron, Fe",40.127059,8.0
"Magnesium, Mg",476.579507,400.0
Niacin,49.165911,16.0


In [18]:
minimum_cost_diet(diet_type='pescatarian', sex_age_group='M 19-30')

Diet in hectograms
Food
Canned Tomatoes    16.685909
Navy Beans          1.241207
Spinach             2.539564
Wheat Cereal        1.940806
dtype: float64

Cost of a pescatarian diet for M 19-30 is $6.49 per day.

Constraining nutrients are:
['Protein', 'Vitamin E (alpha-tocopherol)', 'Sodium, Na']


,Outcome,Recommendation for M 19-30
Nutrition,,
Energy,2400.000173,2400.0
Protein,56.000000,56.0
"Fiber, total dietary",76.370275,33.6
"Folate, DFE",626.162590,400.0
"Calcium, Ca",1014.298205,1000.0
"Carbohydrate, by difference",314.109432,130.0
"Iron, Fe",38.908071,8.0
"Magnesium, Mg",481.992181,400.0
Niacin,49.660805,16.0


In [19]:
minimum_cost_diet(diet_type='vegetarian', sex_age_group='M 19-30')

Diet in hectograms
Food
Canned Tomatoes    16.685906
Navy Beans          1.241207
Spinach             2.539564
Wheat Cereal        1.940808
dtype: float64

Cost of a vegetarian diet for M 19-30 is $6.49 per day.

Constraining nutrients are:
['Protein', 'Vitamin E (alpha-tocopherol)']


,Outcome,Recommendation for M 19-30
Nutrition,,
Energy,2400.000800,2400.0
Protein,56.000000,56.0
"Fiber, total dietary",76.370269,33.6
"Folate, DFE",626.162719,400.0
"Calcium, Ca",1014.298145,1000.0
"Carbohydrate, by difference",314.109581,130.0
"Iron, Fe",38.908081,8.0
"Magnesium, Mg",481.992326,400.0
Niacin,49.660838,16.0


In [20]:
minimum_cost_diet(diet_type='vegan', sex_age_group='M 19-30')

Diet in hectograms
Food
Canned Tomatoes    16.685907
Navy Beans          1.241207
Spinach             2.539564
Wheat Cereal        1.940807
dtype: float64

Cost of a vegan diet for M 19-30 is $6.49 per day.

Constraining nutrients are:
['Protein', 'Vitamin E (alpha-tocopherol)']


,Outcome,Recommendation for M 19-30
Nutrition,,
Energy,2400.000424,2400.0
Protein,56.000001,56.0
"Fiber, total dietary",76.370273,33.6
"Folate, DFE",626.162666,400.0
"Calcium, Ca",1014.298194,1000.0
"Carbohydrate, by difference",314.109474,130.0
"Iron, Fe",38.908073,8.0
"Magnesium, Mg",481.992222,400.0
Niacin,49.660810,16.0


In [21]:
minimum_cost_diet(diet_type='chipotle', sex_age_group='M 19-30')

Fail to find solution.
chipotle diet does not provide any Vitamin B-12!


In [24]:
minimum_cost_diet(diet_type='mcdonalds', sex_age_group='M 19-30')

ValueError: Invalid input for linprog: b_ub must be a 1-D array; b_ub must not have more than one non-singleton dimension and the number of rows in A_ub must equal the number of values in b_ub

### 1.6 #B Is your solution edible?

Turn your minimum cost diet into a meal. Share the recipe on piazza. If making your diet palatable requires adding additional ingredients (e.g. salt or other spices) besure to document this and account for it in the overall cost. (You may want to take pictures or video of preparation and eating). Whose meal is least costly? Whose is most tasty?

### 1.7 #B Meal Reviews

Get the recipe for the meal from some other group(s), and prepare the meal according to their recipe. Provide a review (follow-up to the piazza post with the recipe). How much did the ingredients cost? Would you recommend it to others? Eat it again? Eat it exclusively?

### 1.8 #C Sensitivity of Solution

In general, your solution will depend on relative prices of different kinds of foods. And these prices can be quite volatile; for example, the prices of fresh fruits and vegetables usually exhibit a great deal of seasonal variation. If the price of a particula rkind of food increases by x%, how much does price of diet change, and how does the composition of the diet change? Try and find an nice way to visualize these changes.

### 1.9 #B What is the total cost for population of interest?

You've been interested in a particular population. It is likely to consist of people of different sexes, ages, and levels of activity. How much food would be required to feed the entire population of interest? At what cost? (You may want to go back to the Population Project and look at counts of people by age and sex for different countries).

### 1.10 #B Review of Your Code

### 1.11 #B Review of Another Group's Code

### 1.12 #A Presentation